# Model Creation Walkthrough

Models in `maxlikespy` are implemented as classes that inherit the base Model class.


## Model Requirements

There are three necessary components of a model declaration in `maxlikespy`

* A function `model(self, x)` that returns the functional form of your model, evaluated over space or time
* A list of the model's parameter names **in the same order** as they are unpacked from the solver input `x`
* A function `objective(self, x)` that returns the model's likelihood

Below is the model definition for the single peak gaussian example in the tutorial.



In [ ]:
class Time(Model):

    """Model which contains a time dependent gaussian compenent 
    and an offset parameter.

    """

    def __init__(self, data):
        super().__init__(data)
        self.param_names = ["a_1", "ut", "st", "a_0"]

    def model(self, x):
        a_1, ut, st, a_0 = x
        self.function = (
            (a_1 * np.exp(-np.power(self.t - ut, 2.) / (2 * np.power(st, 2.)))) + a_0)
        
        return self.function
    
    def objective(self, x):
        fun = self.model(x)
        obj = np.sum(self.spikes * (-np.log(fun)) +
                      (1 - self.spikes) * (-np.log(1 - (fun))))
        
        return obj

The only unknown quantity here is `self.t` which is a numpy array describing the experimental time window which is computed by the `DataProcessor` class.

## A More Complicated Model

Given a dataset with some categorical labels on each trial (in the form of a conditions.json file), we can construct a model that tests for sensitivity to these labels. 

Below is the model definition for a gaussian firing field with components related to two different condition labels. 


In [ ]:
class CatSetTime(Model):

    """Model which contains seperate time-dependent gaussian terms per each given category sets.

    """

    def __init__(self, data):
        super().__init__(data)
        self.plot_t = self.t
        self.t = np.tile(self.t, (data["num_trials"], 1))
        self.conditions = data["conditions"]
        self.param_names = ["ut", "st", "a_0", "a_1", "a_2"]


    def model(self, x):
        ut, st, a_0 = x[0], x[1], x[2]
        a_1, a_2 = x[3], x[4]
        c1 = self.conditions[1]
        c2 = self.conditions[2]

        return ((a_1 * c1 * np.exp(-np.power(self.t - ut, 2.) / (2 * np.power(st, 2.)))) + \
            (a_2 * c2 * np.exp(-np.power(self.t - ut, 2.) / (2 * np.power(st, 2.))))) + a_0

    def objective(self, x):
        fun = self.model(x)
        return np.sum((self.spikes * (-np.log(fun)) +
                        (1 - self.spikes) * (-np.log(1 - (fun)))))


Here the objective function is the same, but `model` is quite different. 
The first difference is `self.t = np.tile(self.t, (data["num_trials"], 1))`, this is done because `c1` and `c2` are both vectors of length `num_trials`.

In this instance since the experimental conditions were loaded from disk, they can be accessed using the `data` dict. This dict contains any supplementary information loaded from `spike_info.json`, x-y position for example.

These are just two relatively simple examples, however they outline the general strategy for creating custom models.